In [2]:
# 이 코드에서는 정규식을 이용해 전처리를 수행하고 lexicon을 만들어 봅니다.  
def ngramEojeol(sentence, n=2): # sentence를 받아 어절 단위로 분리해주는 함수
    '''
    입력:     단어1,   단어2,   단어3,  단어4 : 4
    출력(2) : 단어12,  단어23,  단어34 :        3 - n + 1
    출력(3) : 단어123, 단어234         :        2 - n + 1
    '''
    tokens = sentence.split()
    ngram = []
    
    for i in range(len(tokens) - n + 1):
        ngram.append(' '.join(tokens[i:i + n]))    
        
    return ngram

In [3]:
def ngramUmjeol(term, n = 2): # 음절 단위로 구분하는 함수. sentence를 받아 2개(n=2)씩 쪼갠다.

    ngram = []
    
    for i in range(len(term) - n + 1):
        # ngram.append(tokens_ngram[i:i+n]) # 방법1
        # ngram.append(tuple(tokens_ngram[i:i+n])) # 방법2 (튜플로 반환 시 키값을 쓸 수 있음)
        ngram.append(''.join(term[i:i + n])) # 방법3
        
    return ngram

In [5]:
# 이 코드에서는 깨끗한 단어를 추출하기 위해 심화단계의 tokenize를 수행해 봅니다. (NLP를 위한 전처리)
# 그동안 split()만 사용해 왔지만 다른 방법도 적용해 봅니다. 어느 feature가 좋은지 모르므로 최대한 많이 뽑아내는 것이 목적입니다. 

from konlpy.tag import Kkma # 형태소 분석기 중에는 kkma가 성능이 제일 좋음.
ma = Kkma() # 형태소 분석기 인스턴스
sentence = "오늘 미세먼지는 어제 미세먼지보다 나빠요."

# 명사를 뽑으라고 시키면 => 오늘, 미세, 먼지, 어제, 미세, 먼지가 뽑혀야 함.
# ma.pos(sentence) # 형태소가 부착된(태깅된) 형태로 반환
print([token[0] for token in ma.pos(sentence) if token[1].startswith("NN")])
print(ma.nouns(sentence)) # 명사만 뽑기

# BOW => index term, Lexicon(dictionary)로 부르기도 함. 
# 문장 단위 -> 어절 단위 -> 형태소 단위 -> 품사(명사) 단위로 보고, 추가로 Ngram 단위로도 문장을 볼 예정
# 일반적인 전처리 순서 : 토크나이징 -> Normalization(단어의 길이, 한국어의 경우 1음절 단위로 수행하기도 함)
from nltk.tokenize import sent_tokenize, word_tokenize # 두 개의 토큰화 모듈 임포트 

# 구두점에 대한 처리만 빼고 비슷한 결과가 나올 것임.
print(sentence.split()) # 단순히 split
print(word_tokenize(sentence)) # 구두점도 별도로 분류를 했으므로 어절이 누구인지 찾을 수 있다.

print("원본 : ", sentence)

# tokenized data
lexicon = list()
th = 1 # 1음절보다 큰 문자를 고르기 위한 상수
lexicon = [token for token in word_tokenize(sentence) if len(token) > th]
print("어절 단위 분리 : ", lexicon)

# 품사에 대해 태깅 작업 수행
# print(ma.pos(sentence)) # 둘의 차이는 그렇게 크지 않다.
for token in [token for token in word_tokenize(sentence) if len(token) > th]: # 명사 분류시 문제가 될 수 있으므로 토큰화한 후 수행
    lexicon.extend([token[0] for token in ma.pos(token) if len(token[0]) > th]) # (단어, 품사) 튜플쌍으로 받기 위해 for문을 돌면서 형태소 분석. (ma.morphs() 함수를 사용하면 형태소만 반환받을 수 있음. )
    # lexicon.extend([token[0] for token in ma.pos(token) if len(token[0]) > th] and token[1] in ["NN", "NNG"]) 등과 같이 and조건 옆에 원하는 품사를 넣어서 그 POS 태그에 해당하는 것들만 튜플쌍으로 받을 수도 있다. 
print("형태소 분석 결과 : ", list(set(lexicon))) # 필요없는 중복값을 날리기 위해 set()에 담음 

for token in [token for token in word_tokenize(sentence) if len(token) > th]: # 명사 분류시 문제가 될 수 있으므로 토큰화한 후 수행
    lexicon.extend(ma.nouns(token))
print("명사 분석 결과 : ", list(set(lexicon))) # 필요없는 중복값을 날리기 위해 set()에 담음 
# 더 세밀한 분석을 위해 N-gram을 적용할 필요가 있다.

lexicon.extend(ngramEojeol(" ".join([token[0] for token in ma.pos(sentence)])))
print("어절(바이그램) 분석 : ", list(set(lexicon)))

newLexicon = list()
for term in lexicon:
    newLexicon.extend(ngramUmjeol(term))
lexicon.extend([term for term in newLexicon if len(term.strip()) > th])
print("음절(바이그램) 분석 : ", list(set(lexicon)))
print(len(list(set(lexicon))))


['오늘', '미세', '먼지', '미세', '먼지']
['오늘', '미세', '미세먼지', '먼지']
['오늘', '미세먼지는', '어제', '미세먼지보다', '나빠요.']
['오늘', '미세먼지는', '어제', '미세먼지보다', '나빠요', '.']
원본 :  오늘 미세먼지는 어제 미세먼지보다 나빠요.
어절 단위 분리 :  ['오늘', '미세먼지는', '어제', '미세먼지보다', '나빠요']
형태소 분석 결과 :  ['오늘', '아요', '보다', '미세', '나빠요', '미세먼지는', '미세먼지보다', '나쁘', '어제', '먼지']
명사 분석 결과 :  ['오늘', '아요', '보다', '미세', '미세먼지', '나빠요', '미세먼지는', '미세먼지보다', '나쁘', '어제', '먼지']
어절(바이그램) 분석 :  ['오늘', '미세먼지는', '어제 미세', '미세먼지보다', '나쁘', '아요', '미세', '미세 먼지', '나빠요', '보다 나쁘', '먼지 는', '보다', '미세먼지', '오늘 미세', '어제', '나쁘 아요', '먼지', '는 어제', '먼지 보다', '아요 .']
음절(바이그램) 분석 :  ['오늘', '미세먼지는', '어제 미세', '미세먼지보다', '나쁘', '아요', '미세', '미세 먼지', '나빠요', '빠요', '나빠', '세먼', '보다 나쁘', '먼지 는', '지보', '보다', '미세먼지', '오늘 미세', '어제', '나쁘 아요', '먼지', '는 어제', '먼지 보다', '지는', '아요 .']
25


In [6]:
# 색인어 목록
sentence = '오늘 날씨는 어제 날씨보다 안좋은거 같아요'

lexicon = list(set(sentence.split()))
print('1. ', lexicon)

# 색인어 목록 확장(품사, 단어 길이 고려)
for token in [word for word in Kkma().pos(sentence) if word[1] in ['NNG','VA'] and len(word[0]) > 1]:
    lexicon.append(token[0])
    
print('2. ', list(set(lexicon)))

# 어절, 특정 품사&길이 형태소, Ngram(어절)
lexicon.extend(ngramEojeol(sentence))
print('3. ', list(set(lexicon)))

lexicon.extend(ngramEojeol(' '.join([token[0] for token in [word for word in Kkma().pos(sentence) if word[1] in ['NNG','VA'] and len(word[0]) > 1]])))
print('4. ',list(set(lexicon)))

newLexicon = list()
for word in lexicon:
    if len(word.split()) == 1:
        newLexicon.extend(ngramUmjeol(word))
lexicon.extend(newLexicon)
print('5. ', list(set(lexicon)))

1.  ['오늘', '날씨는', '같아요', '어제', '안좋은거', '날씨보다']
2.  ['오늘', '날씨', '같아요', '날씨는', '어제', '안좋은거', '날씨보다']
3.  ['오늘', '어제 날씨보다', '날씨', '안좋은거 같아요', '날씨는 어제', '날씨보다 안좋은거', '오늘 날씨는', '같아요', '날씨는', '어제', '안좋은거', '날씨보다']
4.  ['오늘', '어제 날씨보다', '날씨 어제', '날씨', '안좋은거 같아요', '날씨는 어제', '날씨보다 안좋은거', '오늘 날씨는', '같아요', '날씨는', '오늘 날씨', '어제', '안좋은거', '날씨보다', '어제 날씨']
5.  ['오늘', '은거', '날씨는 어제', '오늘 날씨는', '어제 날씨보다', '안좋', '아요', '날씨보다 안좋은거', '날씨는', '오늘 날씨', '날씨보다', '어제 날씨', '보다', '안좋은거 같아요', '씨는', '같아요', '어제', '안좋은거', '같아', '날씨 어제', '날씨', '좋은', '씨보']


In [7]:
from os import listdir

# path = "C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts/" (상대경로)
# path = "0314_DownloadedNewstxts" (절대경로)

# fileids() => 말뭉치 목록을 리턴. 이 함수처럼 getFileList라는 함수를 만들자.  
def getFileList(base = "./", ext = "txt"): # 아무것도 안했다면 base는 현재 경로
    fileList = list()
    for file in listdir(base):
        if file.split(".")[-1] == ext: # .을 기준으로 split한 것이 txt인지 검사
            fileList.append("{0}/{1}".format(base, file))
            
    return fileList

In [8]:
def getContent(file): # txt 컨텐츠를 편하게 읽어오기 위한 함수
    with open(file, encoding="UTF-8") as f:
        content = f.read()
    return content

In [9]:
len(getFileList("C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts/")) # 정상작동하는지 확인

180

In [10]:
content = getContent(getFileList("C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts/")[0]) # 정상작동하는지 확인

~~~
'\n\n\n\n\n// flash \nfunction _flash_removeCallback() {}\n\n jawon1212@donga.com]\n\n'와 같이 거슬리는 단어를 정규식을 응용해 걸러낼 필요가 있다. 
~~~

In [11]:
import re
from string import punctuation # !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~와 같은 불용어가 들어 있다. 
# 문자를 기입할때는 보통 []로 감싸서 사용. 

# 패턴의 리스트를 돌려주는 함수(key로 pattern을 호출해서 re.sub을 적용하기 위한 목적)
def getPatternList():
    patternList = dict()
    pattern = re.compile(r"[%s]{2,}" % re.escape(punctuation)) # punctuation 안의 특수문자가 두번이상 반복되는 모든 문자에 대해 패턴 정의
    patternList["Punctuation"] = pattern
    pattern2 = re.compile(r"([A-Za-z0-9\-\_\.]{3,}@[A-Za-z0-9\-\_]{3,}(.[A-Za-z]{2,})+)") #이메일주소제거패턴
    patternList["Email"] = pattern2
    pattern3 = re.compile(r"([A-Za-z0-9\-\_]{1,}(.[A-Za-z]{2,})+)") # 신문사도메인패턴
    patternList["Domain"] = pattern3
    pattern4 = re.compile(r"\s{2,}") # 공백제거
    patternList["Whitespace"] = pattern4
    pattern5 = re.compile(r"([^ㄱ-ㅎㅏ-ㅣ가-힣0-9]+)") # 한글이 아닌 영어 기호 제거
    patternList["Korean"] = pattern5
    
    return patternList
    
    
# for _ in ["Email", "Domain", "Nonword", "Punctuation", "Whitespace"]:
    
    
# # 첫번째 필터링 : 특수문자 제거
# pattern = re.compile(r"[%s]{2,}" % re.escape(punctuation)) # punctuation 안의 특수문자가 두번이상 반복되는 모든 문자에 대해 패턴 정의 
# print("------------------------------------------------------------------------------------------------------------------")
# print("1st pattern : ",  pattern) # 패턴이 잘 컴파일되었는지 확인 목적  
# print("1st findall result : ", pattern.findall(content)) # 패턴대로 잘 찾아지는지 확인 목적
# print("1st filtered result : ", pattern.sub(" ", content)) # 정의된 패턴을 통해 2번이상 반복되는 특수 문자를 제거한 결과물(" "으로 치환한 결과) 출력
# print("------------------------------------------------------------------------------------------------------------------")
# 
# # 두번째 필터링 : 이메일 주소 제거
# pattern2 = re.compile(r"([A-Za-z0-9\-\_\.]{3,}@[A-Za-z0-9\-\_]{3,}(.[A-Za-z]{2,})+)") # 이메일은 한글을 사용하지 못하므로 영어, 숫자, 하이픈, . 등만 존재할 것이다. 그 다음 @이 나옴.
# print("2nd pattern : ",  pattern2)
# print("2nd findall result : ", pattern2.search(content)) # 매치되는 딱 하나의 결과만 리턴
# print("2nd filtered result : ", pattern2.sub(" ", content))
# print("------------------------------------------------------------------------------------------------------------------")
# 
# # 세번째 필터링 : 언론사 도메인 주소 제거 (ex. www.khan.co.kr)
# pattern3 = re.compile(r'(\w+@[A-Za-z0-9\-\_]{3,}(.[A-Za-z]{2,})+)') # www.domain.com일 경우, www과 같은 서브도메인이 붙고 후속 내용들이 뒤따르는 형태. 
# print("3rd pattern : ",  pattern3)
# print("3rd findall result : ", pattern3.findall(content)) 
# print("3rd filtered result : ", pattern3.sub(" ", content))
# print("------------------------------------------------------------------------------------------------------------------")
# 
# # 네번째 필터링 : 화이트스페이스 제거
# pattern4 = re.compile(r"\s{2,}") # 공백이 두번이상 반복되는 케이스에 대한 패턴 정의
# print("4th pattern : ",  pattern4)
# print("4th findall result : ", pattern4.findall(content)) 
# print("4th filtered result : ", pattern4.sub(" ", content))
# print("------------------------------------------------------------------------------------------------------------------")
# 
# # 다섯번째 필터링 : 한글이 아닌 영어 및 기호 제거
# pattern5 = re.compile(r"([^ㄱ_ㅎㅏ_ㅣ가_힣0-9]+)")
# print("5th pattern : ",  pattern5)
# print("5th findall result : ", pattern5.findall(content)) 
# print("5th filtered result : ", pattern5.sub(" ", content))
# print("------------------------------------------------------------------------------------------------------------------")

# 한글 이외에 모두 제거
# pattern = re.compile(r'([^ㄱ-ㅎㅏ-ㅣ가-힣]+)')

# 구두점 제거 (2번이상 반복되는 경우)
# pattern = re.compile(r'[%s]{2,}' % re.escape(punctuation)) # re.compile(r'//') 

# 영어 (대소문자) +\-\_ 제거 (8글자 이상)
# pattern = re.compile(r'[A-Za-z-_]{8,}')

# _flash_remoceCallback같은 거슬리는 영어 문구 제거(보통 EMFscientiest와 같이 특별한 의미를 갖고 있는 영단어 때문에 사용하면 위험함)
# pattern = re.compile(r"[A-Za-z\-\_]{4,}") # 4글자 이상 반복되는 영어 문구 찾아내기
# print("pattern : ",  pattern)
# print("findall result : ", pattern.findall(content))
# print("filtered result : ", pattern.sub(" ", content))
# print("------------------------------------------------------------------------------------------------------------------")

In [12]:
patternList = getPatternList()

content = getContent(getFileList("C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts/")[0])
for _ in ["Korean", "Whitespace"]:
    content = patternList[_].sub(" ", content)
print(content)

 오류를 우회하기 위한 함수 추가 오히려 단체측 무선이어폰 연구는 존재치도 않아 국내 참여 연구자도 어리둥절 이엠에프사이언티스에서 일부 언론 보도를 부정하며 직접 보내온 이메일 내용이다 이엠에프사이언티스트 관계자인 조엘 모스코위츠 미국 버클리 캘리포니아대 가정사회건강연구소 소장은 무선 이어폰의 건강 유해성에 대한 보도를 부정했다 고재원 기자 1212 18일 오전 전세계 과학자들이 애플 에어팟과 같은 무선 이어폰이 암 발생 위험을 키울 수 있다는 호소문을 국제연합 과 세계보건기구 에 제출했다는 일부 국내외 언론의 보도가 나오면서 불안감이 확산되고 있다 하지만 실제 이 단체와 호소문에 이름을 올린 과학자들에게 확인한 결과 호소문은 4년전 제출됐던 것이며 또 특정 제품이나 제조사를 언급하지 않은 것으로 확인됐다 18일 데일리메일과 중앙일보 등 국내외 일부 언론에 따르면 전 세계 과학자 247명이 무선 이어폰의 비이온화 전자기장 이 암을 유발할 위험 우려가 있다며 과 에 호소문을 제출했다고 전했다 이들 매체들은 호소문에 애플 에어팟이 에 관한 법적 기준치를 준수하고 있지만 장시간 노출될 경우 건강에 좋지 않을 수 있다 는 내용이 포함됐다고 보도했다 이들 매체들은 이 호소문에는 전 세계 42개국 과학자 247명이 서명을 했다고 전했다 여기에는 한국의 연세대 한양대 가톨릭대 단국대 중앙대 경북대 한림대 소속 과학자 15명의 이름도 포함됐다 하지만 취재 결과 호소문 작성을 주도한 비영리단체 이엠에프사이언티스트 는 애플 에어팟과 같은 무선 이어폰에 대한 유해성을 주장하지 않은 것으로 확인됐다 동아사이언스가 이엠에프사이언티스트에 직접 이메일로 확인한 결과 일부 언론 보도가 호소문에 대한 부정확한 내용을 담고 있다 며 무선 블루투스의 자기장에 머리가 장시간 노출될 시의 안정성에 대한 연구는 존재하지 않는다 고 밝혔다 이엠에프사이언티스트에 따르면 이 단체는 지난 2015년 5월 실제로 전세계 과학자 190명의 서명을 받아 과 유엔환경계획 에 국제 과학자 호소문 을 제출하기는 했다

In [14]:
from nltk.tokenize import sent_tokenize

len(sent_tokenize(content)) # 구두점이 사라져서 토큰화 못함.

dictTerm = list()
dictPos = list()
dictNoun = list()
dictNgram = list()

for sentence in sent_tokenize(content):
    for token in word_tokenize(sentence):
        if len(token) > th:
            dictTerm.append(token)
            # 아래서부터는 list이기때문에 extend를 사용해야 함. 
            dictPos.extend([morpheme for morpheme in ma.morphs(token) if len(morpheme) > th]) # 형태소 분석결과를 extend
            dictNoun.extend([noun for noun in ma.nouns(token) if len(noun) > th]) # 명사 단위로 잘라 extend
            dictNgram.extend(ngramUmjeol(token)) # 바이그램을 리턴
            
# 빠른 속도 및 중복 데이터 처리를 위해 set사용
dictTerm = list(set(dictTerm))
dictPos = list(set(dictPos))
dictNoun = list(set(dictNoun))
dictNgram = list(set(dictNgram))

In [15]:
len(dictTerm), len(dictPos), len(dictNoun), len(dictNgram) # (282, 219, 188, 492)

# len(termList) : 순수하게 어절 단위로 자른 차원 수를 알 수 있고,(282차원)
# th(길이)가 1보다 큰것들로 분류했으므로 은, 는, 이, 가와 같은 문자들이 사라졌을것. 
# len(posList), len(nounList)는 큰 차이가 없음. 즉 한국어는 명사 위주로 중요한 내용을 표현함을 알 수 있다. 
# ngram 모델의 단점 중 space complexity가 높다는 문제를 len(ngramList)을 통해 알 수 있다. 

(283, 220, 189, 494)

In [16]:
# 문서가 크면 클수록 계속해서 선형적으로 늘어난다. 이런식으로 뽑아낸 대량의 lexicon을 두고 query에 대해 잘 retrieval하는 것이 목적. 
len(list(set(dictTerm + dictPos + dictNoun + dictNgram))) # 최종적으로 뽑은 term set (lexicon, index term list라고 부름.)

767

In [17]:
print(dictTerm)

['5월', '없다고', '247명으로', '서명했던', '관계자인', '발상도', '우리', '이어폰에', '주장하지', '확인됐다', '있는데', '최근', '언론', '블루투스의', '이어폰이', '한국의', '송기원', '존재치도', '포함됐다고', '담았다', '인체에', '않았고', '18일', '필요하다는', '통해', '지나다니고', '명단이', '이엠에프사이언티스', '생화학과', '서명한', '머리가', '국내', '연구를', '주도한', '올라간', '위험', '마이크로파가', '버클리', '유해성을', '이엠에프사이언티스트', '직접', '2000개가', '이엠에프사이언티스에서', '불안감이', '15명도', '보호와', '통신용', '유해성', '호소문에는', '서명을', '포함됐다', '가져간다고', '단국대', '있지', '호소문은', '이덕환', '과학자들에게', '대략', '과학자들은', '결론내렸다', '미국', '한림대', '않는다', '있다며', '서명했다고', '가톨릭대', '해당', '현재는', '건강과', '부정하며', '분들', '언론에', '자체', '오류를', '있지만', '여부는', '가까이', '제출했다고', '이들', '이라며', '과학자들의', '함수', '당시', '법적', '이메일', '받아', '준수하고', '방지가', '담고', '알려진', '된다는', '주변에', '과학자들이', '위험을', '무선', '노출로부터', '연구들을', '있다', '이메일로', '전자기장과', '이름을', '것으로', '서명자인', '키울', '2만5000개가', '기자', '한양대', '어리둥절', '한국인', '30년간', '않을', '애플', '확인하지', '않아', '42개국', '오전', '2년전', '넘는', '연구는', '세계', '전했다', '동아사이언스가', '이미', '무선이어폰을', '보도를', '했다', '영향을', '제출했다는', '기억이', '가정사회건강연구소', '이어폰의', '올린', '제조

In [18]:
print(dictPos)

['유발', '우리', '계획', '이나', '15', '블루투스', '우려', '한림', '화학', '사회', '최근', '언론', '부정', '언급', '서명', '자인', '송기원', '제조사', '강대', '문제', '성에', '국내', '통하', '연합', '저명', '위험', '5000', '결론', '기구', '기억', '논란', '버클리', '이어폰', '직접', '보호', '유해성', '개국', '필요', '노출', '올라가', '매체', '부정확', '다며', '명단', '존재', '에게', '대략', '소장', '미국', '인터뷰', '가톨릭대', '해당', '자체', '이번', '지방', '가까이', '방지', '학술지', '함수', '가정', '근거', '당시', '법적', '이메일', '지속적', '동의', '게재', '확인', '주제', '190', '2000', '머리', '주도', '무선', '연구', '단체', '보내오', '247', '비이온', '한국', '기자', '한국인', '어리둥절', '메일', '애플', '오전', 'ㄴ다는', '티스', '관여', '인체', '유엔', '아니', '세계', '이미', '국립대', '미치', '모르', '다는', '소속', '못하', '스트', '참여', '전세계', '중앙대', '하다', '라며', '중앙일보', '실제', '보도', '에어팟', 'ㄴ다고', '18', '따르', '30', '사이', '추가', '양대', '키우', '학과', '으로', '관계자', '결과', '비영리', '수많', '밝히', '국제', '여기', '나오', '오히려', '발생', '포함', '시의', '모스코', '말하', '기준치', '자기장', '지나', '과학자', '특정', '불안감', '우회', '실제로', '동아', '주장', '비전', '건강', '42', '2015', '안정성', '현재', '주변', '제품', '는데', '장과', '오류', '발표', '일부', '사이언스', '준

In [19]:
print(dictNoun)

['5월', '유발', '우리', '계획', '15', '블루투스', '한림', '화학', '사회', '최근', '언론', '부정', '언급', '서명', '자인', '송기원', '18일', '제조사', '데일리메일', '성에', '생화학과', '국내', '연합', '저명', '위험', '결론', '기구', '기억', '논란', '버클리', '이어폰', '보호', '통신용', '유해성', '개국', '필요', '노출', '매체', '부정확', '명단', '존재', '소장', '미국', '인터뷰', '가톨릭대', '필요하다', '해당', '자체', '이번', '지방', '학술지', '함수', '가정', '근거', '당시', '법적', '이메일', '지속적', '동의', '게재', '확인', '주제', '190', '주도', '무선', '연구', '단체', '247', '비이온', '전자기장과', '한국', '서명자인', '기자', '한국인', '메일', '30년간', '덕환', '애플', '42개국', '오전', '2년전', '티스', '관여', '인체', '유엔', '15명', '세계', '국립대', '가정사회건강연구소', '소속', '스트', '참여', '전세계', '중앙대', '하다', '중앙일보', '실제', '보도', '에어팟', '18', '190명', '30', '사이', '모스코위츠', '추가', '지방국립대', '티스트', '학과', '비영리단체', '관계자', '결과', '비영리', '전자기장의', '국제', '여기', '발생', '포함', '시의', '모스코', '4년전', '기준치', '자기장', '과학자', '특정', '247명', '불안감', '우회', '주장', '비전', '엠에프사이언', '일보', '건강', '42', '2015', '안정성', '현재', '주변', '제품', '장과', '오류', '발표', '일부', '준수', '발상', '데일리', '중앙', '전화', '재원', '작성', '취재', '2015년', '내용', 

In [20]:
print(dictNgram)

['도한', '우리', '파가', '인터', '화학', '부정', '장시', '것으', '성에', '국내', '수가', '연합', '2년', '증됐', '통신', '제연', '명했', '의하', '개가', '제조', '둥절', '있지', '존재', '대략', '학술', '않는', '랐다', '사를', '자체', '0개', '강에', '이들', '방지', '팟이', '림대', '게재', '투스', '담고', '장에', '주제', '치는', '주도', '연구', '앙일', '번에', '하며', '름을', '키울', '이온', '이크', '50', '련된', '티스', '릭대', '유엔', '수는', '세계', '모스', '직까', '라간', '뷰에', '아닌', '가져', '하지', '도했', '정사', '인하', '라며', '립대', '했던', '지에', '18', '30', '만5', '사이', '됐으', '추가', '준치', '전했', '여기', '어났', '출될', '치도', '없다', '연세', '주장', '주변', '강과', '내렸', '시간', '세대', '블루', '못했', '작성', '체측', '같은', '버클', '구자', '감이', '난다', '아직', '황당', '제출', '도를', '들을', '안정', '방국', '않은', '니아', '대한', '명이', '폰을', '론내', '확산', '올라', '일로', '비영', '오면', '여부', '유해', '성을', '름도', '폰이', '류를', '어팟', '낮은', '관련', '향을', '가까', '출된', '국의', '계획', '이나', '산되', '우려', '한림', '났다', '최근', '용이', '서명', '년간', '어리', '혔다', '론의', '변에', '말했', '앙대', '들이', '북대', '기억', '리가', '부는', '많은', '된다', '이며', '명단', '이엠', '치를', '체에', '이번', '술지', '근거', '일리', '동의', '명의', '명한', '관계